# Westeros Tutorial
## Adding fossil resources

In this tutorial we will cover the addition of fossil resources, i.e. resource supply curves, to the model.  This will include defining a `resource_volume` and respective costs, as well as the specifying what share of these resources must be preserved over time.  Multiple resource categories can be added for a single commodity, allowing, for example, a differentiation to be made between '*reserves*' and '*resources*' and/or the location of the resources, i.e. above or below ground; on- or off-shore.

‘*Reserves*’ are generally defined as being those quantities for which geological and engineering information indicate with reasonable certainty that they can be recovered in the future from known reservoirs under existing economic and operating conditions. ‘*Resources*’ are detected quantities that cannot be profitably recovered with current technology, but might be recoverable in the future, as well as those quantities that are geologically possible, but yet to be found. Definitions are based on Rogner et al. (2012). 

In this tutorial, we will be adding two categories of coal resources.  The assumed potential is based on the coal requirements by the `coal_ppl` in the baseline, and will be split so that we can also observe the use of multiple resource categories. Further, we will also add a coal extraction technology, linking the coal resources to the primary energy level.

In [ ]:
import pandas as pd
import ixmp as ix
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [ ]:
mp = ix.Platform(dbtype='HSQLDB')

## Load the existing scenario '*carbon_tax*' and clone to a new scenario '*renewable_potential*' to which we will apply the `renewable_resource_constraints` constraint

In [ ]:
model = 'Westeros Electrified'
base = message_ix.Scenario(mp, model=model, scenario='baseline')
scen = base.clone(model, 'fossil_resources', 'illustration of adding fossil resources', keep_solution=False)
scen.check_out()

## Retrieve parameters to perform subsequent addition of parameters and define 'base' dataframes for adding parameters

In [ ]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

base_input = {
    'node_loc': country,
    'year_vtg': vintage_years
    ,
    'year_act': act_years,
    'mode': 'standard',
    'node_origin': country,
    'commodity': 'electricity',
    'time': 'year',
    'time_origin': 'year',
}

base_output = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'node_dest': country,
    'time': 'year',
    'time_dest': 'year', 
    'unit': '%',
}

base_capacity_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'time': 'year',
    'unit': '%',
}

base_technical_lifetime = {
    'node_loc': country,
    'year_vtg': model_horizon,
    'unit': 'y',
}

base_inv_cost = {
    'node_loc': country,
    'year_vtg': model_horizon,
    'unit': 'USD/GWa',
}

base_fix_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'unit': 'USD/GWa',
}

base_var_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'time': 'year',
    'unit': 'USD/GWa',
}

## `resource_potential` and `grade`- Describing the fossil resource potentials

Introducing fossil resources requires the following steps to be carried out:
1. Add level and commodity required for resources:
   - Specify the new level and commodity which accounts for the coal resources.
   - Specify which level is a `resource`
2. Add potentials and corresponding parameters
   - Add `grade`
   - Add `resource_volume`
   - Add `resource_remaining`
   - Add `resource_cost`
   - Add `historical_extraction`
3. Add an extraction technology for coal
4. Modify existing `coal_ppl` technology by adding a new `input` parameter.

For the purpose of determining the required amount of coal for our fictional example, we will retrieve the activity level of the `coal_ppl` in the '*baseline*' scenario. As the activity is an annual value, we will need to multiply the annual values by the respective period length using the parameter `duration_period`. By summing up the resulting values over the model time horizon, we can determine the cumulative the required amount of coal by the `coal_ppl`.

In [ ]:
# Retrieve activity from the baseline scenario for the `coal_ppl`
# and modify dataframe so it can be multiplies by the period length
coal_ppl_act = base.var('ACT', filters={'technology': ['coal_ppl']})\
                   .drop(['time', 'mrg', 'year_vtg'], axis=1)\
                   .groupby(['node_loc', 'technology', 'mode', 'year_act']).sum()\
                   .reset_index()\
                   .set_index('year_act')
coal_ppl_act

In [ ]:
# Retrieve the parameter `duration_period` to determine the period length
year_length = base.par('duration_period')\
                  .drop(['unit'], axis=1)\
                  .rename(columns={'year': 'year_act',
                                   'value': 'lvl'})\
                  .set_index('year_act')
year_length

In [ ]:
# multiply the activity of the coal_ppl by the period length and sum over all years
coal_ppl_act['lvl'] *= year_length['lvl']
coal_ppl_act['lvl'].sum()

We now know that in the '*baseline*' scenario the `coal_ppl` required 3942 GWa of coal.  We will set the potentials to a slightly lower amount so that we can clearly see the impact on the model results. In addition, we will also define the parameter `resource_remaining`.  This parameter defines the share of resources that can be extracted in relation to the total potential specified for a certain grade, i.e. restricting the model so that it cannot use 100% resources in a single timestep.  This is similar to the aim of a market diffusion constraint for technologies; also when the market price is 10\$, and two resources cost 2\$ and 4\$ respectively, then the model would only choose the cheepest, whereas in reality both would be extracted.

In [ ]:
coal_ppl_hact = base.par('historical_activity', filters={'technology': ['coal_ppl']})
coal_ppl_hact

In [ ]:
# {grade: [volume, cost, share remaining, historical_extraction]}
#potentials = {'a': [1000, 1.0, .3, 280],
#              'b': [2942, 6.0, .3, 0]}

potentials = {'a': [1000, 1.0, .6, 280],
              'b': [2942, 6.0, .6, 0]}

In [ ]:
commodity= 'coal'
level = 'resource'
scen.add_set('commodity', commodity)
scen.add_set('level', level)
scen.add_set('level_resource', level)
for grade in potentials:
    scen.add_set('grade', grade)
    # index for resource_volume is ['node', 'commodity', 'grade', 'value', 'unit']
    df = pd.DataFrame({'node': [country],
                       'commodity': commodity,
                       'grade': grade,
                       'value': potentials[grade][0],
                       'unit': 'GWa'})
    scen.add_par('resource_volume', df)
    
    # index for resource_remaining is ['node', 'commodity', 'grade', 'year', 'value', 'unit']
    df = pd.DataFrame({'node': country,
                       'commodity': commodity,
                       'grade': grade,
                       'year': model_horizon.values.tolist(),
                       'value': potentials[grade][2],
                       'unit': 'GWa'})
    scen.add_par('resource_remaining', df)
    
    # index for resouce_cost is ['node', 'commodity', 'grade', 'year', 'value', 'unit']
    df = pd.DataFrame({'node': country,
                       'commodity': commodity,
                       'grade': grade,
                       'year': model_horizon.values.tolist(),
                       'value': potentials[grade][1],
                       'unit': 'GWa'})
    scen.add_par('resource_cost', df)
    
    # index for historical_extraction is ['node', 'commodity', 'grade', 'year', 'value', 'unit']
    df = pd.DataFrame({'node': country,
                       'commodity': commodity,
                       'grade': grade,
                       'year': [690],
                       'value':  potentials[grade][3],
                       'unit': 'GWa'})
    scen.add_par('historical_extraction', df)

In [ ]:
level = 'primary'
scen.add_set('level', level)

tec = 'coal_extr'
scen.add_set('technology', tec)

df = make_df(base_output, technology=tec, commodity=commodity, 
                   level=level, value=1)
scen.add_par('output', df)

df = make_df(base_input, technology=tec, commodity=commodity, 
                   level='resource', value=1, unit='%')
scen.add_par('input', df)

df = make_df(base_capacity_factor, technology=tec, value=1)
scen.add_par('capacity_factor', df)

df = make_df(base_technical_lifetime, technology=tec, value=10)
scen.add_par('technical_lifetime', df)

df = make_df(base_inv_cost, technology=tec, value=10)
scen.add_par('inv_cost', df)

df = make_df(base_fix_cost, technology=tec, value=2)
scen.add_par('fix_cost', df)

df = make_df(base_var_cost, technology=tec, value=2)
scen.add_par('var_cost', df)

In [ ]:
df = make_df(base_input, technology='coal_ppl', commodity=commodity, 
                   level=level, value=1, unit='%')
scen.add_par('input', df)

### Commit and solve

In [ ]:
scen.commit(comment='added coal resources')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

In [ ]:
base.var('ACT', filters={'technology': ['coal_ppl']})

In [ ]:
scen.var('ACT', filters={'technology': ['coal_ppl']})

# Plotting Results

In [ ]:
from tools import Plots
p = Plots(scen, country, firstyear=700) # scenario: 'renewable_potential' (emission_bound scenario with firm capacity)
b = Plots(base, country, firstyear=700) # scenario: 'carbon_tax' (without renewable potentials)

## Activity
***
When comparing the results of the original scenario without the renewable potentials ('*carbon_tax*') with the results of our newly modified scenario ('*renewable_potential*'), for the same carbon price we can observe that the activity of the `wind_ppl` has substantially decreased.  This is because through adding potentials with corresponding plant factors, the `wind_ppl` has become increasingly economicaly unattractive and despite the carbon tax is not used. 
Note, that the `coal_ppl` still has a plant factor of 1 and has no resource constraints, thus in order to further improve the model, the parameters for the `coal_ppl` would need to be adjusted.

### Scenario: '*carbon_tax*'

In [ ]:
b.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

### Scenario: '*renewable_potential*'

In [ ]:
p.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

## Capacity
***
The behavior observed for the activity of the two electricity generation technologies is reflected in the capacity. No further capacity is built for the `wind_ppl` and thus is phased out by 720.

### Scenario: '*carbon_tax*'

In [ ]:
b.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

### Scenario: '*renewable_potential*'

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

## Prices
***
Especially in the earlier model time periods, electricity and therefore the price for light increase dramatically.  The increase in 720 is due to the emission taxes associated with the operation of the `coal_ppl`.

### Scenario: '*carbon_tax*'

In [ ]:
b.plot_prices(subset=['light'], baseyear=True)

### Scenario: '*renewable_potential*'

In [ ]:
p.plot_prices(subset=['light'], baseyear=True)

In [ ]:
mp.close_db()